In [1]:
import pandas as pd
import numpy as np

# === CONFIGURE ETF ALLOCATION PERCENTAGES ===
ABS_allocation = 95.07  # % in CLO/ABS
MMF_allocation = 2.34   # % in MMF or other

# === Define rating mapping (Moody's and S&P) to 1–4 scale ===
def map_agency_rating(r):
    r = str(r).strip().upper()
    # Moody's
    if r in ['AAA', 'AAA', 'AAA', 'AAA', 'AAA', 'AAA']: return 1
    if r in ['AA1', 'AA2', 'AA3', 'AA+', 'AA', 'AA-']: return 1
    if r in ['A1', 'A2', 'A3', 'A+', 'A', 'A-']: return 2
    if r in ['BAA1', 'BAA2', 'BAA3', 'BBB+', 'BBB', 'BBB-']: return 3
    if r in ['BA1', 'BA2', 'BA3', 'BB+', 'BB', 'BB-', 'B+', 'B', 'B-', 'C', 'CA', 'CAA', 'CCC+', 'CCC', 'CCC-', 'CC', 'D']: return 4
    return np.nan

# === Load Excel file ===
file_path = "PAAA_CLP.xlsx"
df = pd.read_excel(file_path)

# Rename for consistency
df.columns = ['Security', '%Net', 'BVAL', 'CCC', 'Default', 'ManualRating', 'Rating_Agency']

# Score functions
def bval_score(x):
    if x >= 97.5: return 1
    elif x >= 95: return 2
    elif x >= 92: return 3
    else: return 4

def ccc_score(x):
    if x < 5: return 1
    elif x <= 7.5: return 2
    elif x <= 10: return 3
    else: return 4

def default_score(x):
    if x < 0.1: return 1
    elif x < 0.5: return 2
    elif x < 1.0: return 3
    else: return 4


# Score base parameters
df['BVAL_Score'] = df['BVAL'].apply(lambda x: bval_score(x) if not pd.isna(x) else np.nan)
df['CCC_Score'] = df['CCC'].apply(lambda x: ccc_score(x) if not pd.isna(x) else np.nan)
df['Default_Score'] = df['Default'].apply(lambda x: default_score(x) if not pd.isna(x) else np.nan)

# Score agency rating
df['Agency_Score'] = df['Rating_Agency'].apply(map_agency_rating)

# Composite score (4-factor)
def compute_composite(row):
    # If all three base scores are missing → use ManualRating (for ETF-in-ETF)
    if pd.isna(row['BVAL_Score']) and pd.isna(row['CCC_Score']) and pd.isna(row['Default_Score']):
        return row['ManualRating']
    else:
        return (
            0.3 * row['CCC_Score'] +
            0.3 * row['BVAL_Score'] +
            0.2 * row['Default_Score'] +
            0.2 * row['Agency_Score']
        )

df['Composite_Score'] = df.apply(compute_composite, axis=1)

# Risk category
def risk_category(score):
    if score <= 1.75: return 'Benign'
    elif score <= 2.5: return 'Average'
    elif score <= 3.25: return 'Stressed'
    else: return 'Crisis'

df['Category'] = df['Composite_Score'].apply(risk_category)

# Normalize weights
df['Normalized_Weight'] = df['%Net'] / df['%Net'].sum()

# ETF score (CLO-only holdings)
etf_clo_score = (df['Normalized_Weight'] * df['Composite_Score']).sum()

# Adjusted ETF score (includes MMF, assumed benign)
etf_adjusted_score = (
    etf_clo_score * (ABS_allocation / 100) +
    1.0 * (MMF_allocation / 100)
)
etf_rating = risk_category(etf_adjusted_score)

# Final Output
category_counts = df['Category'].value_counts().reindex(['Benign', 'Average', 'Stressed', 'Crisis']).fillna(0).astype(int)

print("CLOs per Category:")
print(category_counts)

print(f"\nUnadjusted CLO-only ETF Score: {etf_clo_score:.2f}")
print(f"Adjusted ETF Score (with MMF): {etf_adjusted_score:.2f}")
print(f"Final ETF Rating: {etf_rating}")


CLOs per Category:
Category
Benign      9
Average     5
Stressed    3
Crisis      3
Name: count, dtype: int32

Unadjusted CLO-only ETF Score: 1.52
Adjusted ETF Score (with MMF): 1.47
Final ETF Rating: Benign


In [3]:
import pandas as pd
import numpy as np

# === CONFIGURE ETF ALLOCATION PERCENTAGES ===
ABS_allocation = 95.07  # % in CLO/ABS
MMF_allocation = 2.34   # % in MMF or other

# === Define rating mapping (Moody's and S&P) to 1–4 scale ===
def map_agency_rating(r):
    r = str(r).strip().upper()
    # Moody's
    if r in ['AAA', 'AAA', 'AAA', 'AAA', 'AAA', 'AAA']: return 1
    if r in ['AA1', 'AA2', 'AA3', 'AA+', 'AA', 'AA-']: return 1
    if r in ['A1', 'A2', 'A3', 'A+', 'A', 'A-']: return 2
    if r in ['BAA1', 'BAA2', 'BAA3', 'BBB+', 'BBB', 'BBB-']: return 3
    if r in ['BA1', 'BA2', 'BA3', 'BB+', 'BB', 'BB-', 'B+', 'B', 'B-', 'C', 'CA', 'CAA', 'CCC+', 'CCC', 'CCC-', 'CC', 'D']: return 4
    return np.nan

# === Load Excel file ===
file_path = "CLOA_CLP.xlsx"
df = pd.read_excel(file_path)

# Rename for consistency
df.columns = ['Security', '%Net', 'BVAL', 'CCC', 'Default', 'ManualRating', 'Rating_Agency']

# Score functions
def bval_score(x):
    if x >= 97.5: return 1
    elif x >= 95: return 2
    elif x >= 92: return 3
    else: return 4

def ccc_score(x):
    if x < 5: return 1
    elif x <= 7.5: return 2
    elif x <= 10: return 3
    else: return 4

def default_score(x):
    if x < 0.1: return 1
    elif x < 0.5: return 2
    elif x < 1.0: return 3
    else: return 4


# Score base parameters
df['BVAL_Score'] = df['BVAL'].apply(lambda x: bval_score(x) if not pd.isna(x) else np.nan)
df['CCC_Score'] = df['CCC'].apply(lambda x: ccc_score(x) if not pd.isna(x) else np.nan)
df['Default_Score'] = df['Default'].apply(lambda x: default_score(x) if not pd.isna(x) else np.nan)

# Score agency rating
df['Agency_Score'] = df['Rating_Agency'].apply(map_agency_rating)

# Composite score (4-factor)
def compute_composite(row):
    # If all three base scores are missing → use ManualRating (for ETF-in-ETF)
    if pd.isna(row['BVAL_Score']) and pd.isna(row['CCC_Score']) and pd.isna(row['Default_Score']):
        return row['ManualRating']
    else:
        return (
            0.3 * row['CCC_Score'] +
            0.3 * row['BVAL_Score'] +
            0.2 * row['Default_Score'] +
            0.2 * row['Agency_Score']
        )

df['Composite_Score'] = df.apply(compute_composite, axis=1)

# Risk category
def risk_category(score):
    if score <= 1.75: return 'Benign'
    elif score <= 2.5: return 'Average'
    elif score <= 3.25: return 'Stressed'
    else: return 'Crisis'

df['Category'] = df['Composite_Score'].apply(risk_category)

# Normalize weights
df['Normalized_Weight'] = df['%Net'] / df['%Net'].sum()

# ETF score (CLO-only holdings)
etf_clo_score = (df['Normalized_Weight'] * df['Composite_Score']).sum()

# Adjusted ETF score (includes MMF, assumed benign)
etf_adjusted_score = (
    etf_clo_score * (ABS_allocation / 100) +
    1.0 * (MMF_allocation / 100)
)
etf_rating = risk_category(etf_adjusted_score)

# Final Output
category_counts = df['Category'].value_counts().reindex(['Benign', 'Average', 'Stressed', 'Crisis']).fillna(0).astype(int)

print("CLOs per Category:")
print(category_counts)

print(f"\nUnadjusted CLO-only ETF Score: {etf_clo_score:.2f}")
print(f"Adjusted ETF Score (with MMF): {etf_adjusted_score:.2f}")
print(f"Final ETF Rating: {etf_rating}")


CLOs per Category:
Category
Benign      17
Average      3
Stressed     0
Crisis       0
Name: count, dtype: int32

Unadjusted CLO-only ETF Score: 1.45
Adjusted ETF Score (with MMF): 1.40
Final ETF Rating: Benign


In [5]:
import pandas as pd
import numpy as np

# === CONFIGURE ETF ALLOCATION PERCENTAGES ===
ABS_allocation = 95.07  # % in CLO/ABS
MMF_allocation = 2.34   # % in MMF or other

# === Define rating mapping (Moody's and S&P) to 1–4 scale ===
def map_agency_rating(r):
    r = str(r).strip().upper()
    # Moody's
    if r in ['AAA', 'AAA', 'AAA', 'AAA', 'AAA', 'AAA']: return 1
    if r in ['AA1', 'AA2', 'AA3', 'AA+', 'AA', 'AA-']: return 1
    if r in ['A1', 'A2', 'A3', 'A+', 'A', 'A-']: return 2
    if r in ['BAA1', 'BAA2', 'BAA3', 'BBB+', 'BBB', 'BBB-']: return 3
    if r in ['BA1', 'BA2', 'BA3', 'BB+', 'BB', 'BB-', 'B+', 'B', 'B-', 'C', 'CA', 'CAA', 'CCC+', 'CCC', 'CCC-', 'CC', 'D']: return 4
    return np.nan

# === Load Excel file ===
file_path = "CLOI_CLP.xlsx"
df = pd.read_excel(file_path)

# Rename for consistency
df.columns = ['Security', '%Net', 'BVAL', 'CCC', 'Default', 'ManualRating', 'Rating_Agency']

# Score functions
def bval_score(x):
    if x >= 97.5: return 1
    elif x >= 95: return 2
    elif x >= 92: return 3
    else: return 4

def ccc_score(x):
    if x < 5: return 1
    elif x <= 7.5: return 2
    elif x <= 10: return 3
    else: return 4

def default_score(x):
    if x < 0.1: return 1
    elif x < 0.5: return 2
    elif x < 1.0: return 3
    else: return 4


# Score base parameters
df['BVAL_Score'] = df['BVAL'].apply(lambda x: bval_score(x) if not pd.isna(x) else np.nan)
df['CCC_Score'] = df['CCC'].apply(lambda x: ccc_score(x) if not pd.isna(x) else np.nan)
df['Default_Score'] = df['Default'].apply(lambda x: default_score(x) if not pd.isna(x) else np.nan)

# Score agency rating
df['Agency_Score'] = df['Rating_Agency'].apply(map_agency_rating)

# Composite score (4-factor)
def compute_composite(row):
    # If all three base scores are missing → use ManualRating (for ETF-in-ETF)
    if pd.isna(row['BVAL_Score']) and pd.isna(row['CCC_Score']) and pd.isna(row['Default_Score']):
        return row['ManualRating']
    else:
        return (
            0.3 * row['CCC_Score'] +
            0.3 * row['BVAL_Score'] +
            0.2 * row['Default_Score'] +
            0.2 * row['Agency_Score']
        )

df['Composite_Score'] = df.apply(compute_composite, axis=1)

# Risk category
def risk_category(score):
    if score <= 1.75: return 'Benign'
    elif score <= 2.5: return 'Average'
    elif score <= 3.25: return 'Stressed'
    else: return 'Crisis'

df['Category'] = df['Composite_Score'].apply(risk_category)

# Normalize weights
df['Normalized_Weight'] = df['%Net'] / df['%Net'].sum()

# ETF score (CLO-only holdings)
etf_clo_score = (df['Normalized_Weight'] * df['Composite_Score']).sum()

# Adjusted ETF score (includes MMF, assumed benign)
etf_adjusted_score = (
    etf_clo_score * (ABS_allocation / 100) +
    1.0 * (MMF_allocation / 100)
)
etf_rating = risk_category(etf_adjusted_score)

# Final Output
category_counts = df['Category'].value_counts().reindex(['Benign', 'Average', 'Stressed', 'Crisis']).fillna(0).astype(int)

print("CLOs per Category:")
print(category_counts)

print(f"\nUnadjusted CLO-only ETF Score: {etf_clo_score:.2f}")
print(f"Adjusted ETF Score (with MMF): {etf_adjusted_score:.2f}")
print(f"Final ETF Rating: {etf_rating}")


CLOs per Category:
Category
Benign       1
Average     17
Stressed     2
Crisis       0
Name: count, dtype: int32

Unadjusted CLO-only ETF Score: 2.18
Adjusted ETF Score (with MMF): 2.09
Final ETF Rating: Average


In [7]:
import pandas as pd
import numpy as np

# === CONFIGURE ETF ALLOCATION PERCENTAGES ===
ABS_allocation = 95.07  # % in CLO/ABS
MMF_allocation = 2.34   # % in MMF or other

# === Define rating mapping (Moody's and S&P) to 1–4 scale ===
def map_agency_rating(r):
    r = str(r).strip().upper()
    # Moody's
    if r in ['AAA', 'AAA', 'AAA', 'AAA', 'AAA', 'AAA']: return 1
    if r in ['AA1', 'AA2', 'AA3', 'AA+', 'AA', 'AA-']: return 1
    if r in ['A1', 'A2', 'A3', 'A+', 'A', 'A-']: return 2
    if r in ['BAA1', 'BAA2', 'BAA3', 'BBB+', 'BBB', 'BBB-']: return 3
    if r in ['BA1', 'BA2', 'BA3', 'BB+', 'BB', 'BB-', 'B+', 'B', 'B-', 'C', 'CA', 'CAA', 'CCC+', 'CCC', 'CCC-', 'CC', 'D']: return 4
    return np.nan

# === Load Excel file ===
file_path = "CLOX_CLP.xlsx"
df = pd.read_excel(file_path)

# Rename for consistency
df.columns = ['Security', '%Net', 'BVAL', 'CCC', 'Default', 'ManualRating', 'Rating_Agency']

# Score functions
def bval_score(x):
    if x >= 97.5: return 1
    elif x >= 95: return 2
    elif x >= 92: return 3
    else: return 4

def ccc_score(x):
    if x < 5: return 1
    elif x <= 7.5: return 2
    elif x <= 10: return 3
    else: return 4

def default_score(x):
    if x < 0.1: return 1
    elif x < 0.5: return 2
    elif x < 1.0: return 3
    else: return 4


# Score base parameters
df['BVAL_Score'] = df['BVAL'].apply(lambda x: bval_score(x) if not pd.isna(x) else np.nan)
df['CCC_Score'] = df['CCC'].apply(lambda x: ccc_score(x) if not pd.isna(x) else np.nan)
df['Default_Score'] = df['Default'].apply(lambda x: default_score(x) if not pd.isna(x) else np.nan)

# Score agency rating
df['Agency_Score'] = df['Rating_Agency'].apply(map_agency_rating)

# Composite score (4-factor)
def compute_composite(row):
    # If all three base scores are missing → use ManualRating (for ETF-in-ETF)
    if pd.isna(row['BVAL_Score']) and pd.isna(row['CCC_Score']) and pd.isna(row['Default_Score']):
        return row['ManualRating']
    else:
        return (
            0.3 * row['CCC_Score'] +
            0.3 * row['BVAL_Score'] +
            0.2 * row['Default_Score'] +
            0.2 * row['Agency_Score']
        )

df['Composite_Score'] = df.apply(compute_composite, axis=1)

# Risk category
def risk_category(score):
    if score <= 1.75: return 'Benign'
    elif score <= 2.5: return 'Average'
    elif score <= 3.25: return 'Stressed'
    else: return 'Crisis'

df['Category'] = df['Composite_Score'].apply(risk_category)

# Normalize weights
df['Normalized_Weight'] = df['%Net'] / df['%Net'].sum()

# ETF score (CLO-only holdings)
etf_clo_score = (df['Normalized_Weight'] * df['Composite_Score']).sum()

# Adjusted ETF score (includes MMF, assumed benign)
etf_adjusted_score = (
    etf_clo_score * (ABS_allocation / 100) +
    1.0 * (MMF_allocation / 100)
)
etf_rating = risk_category(etf_adjusted_score)

# Final Output
category_counts = df['Category'].value_counts().reindex(['Benign', 'Average', 'Stressed', 'Crisis']).fillna(0).astype(int)

print("CLOs per Category:")
print(category_counts)

print(f"\nUnadjusted CLO-only ETF Score: {etf_clo_score:.2f}")
print(f"Adjusted ETF Score (with MMF): {etf_adjusted_score:.2f}")
print(f"Final ETF Rating: {etf_rating}")


CLOs per Category:
Category
Benign      16
Average      4
Stressed     0
Crisis       0
Name: count, dtype: int32

Unadjusted CLO-only ETF Score: 1.45
Adjusted ETF Score (with MMF): 1.40
Final ETF Rating: Benign


In [9]:
import pandas as pd
import numpy as np

# === CONFIGURE ETF ALLOCATION PERCENTAGES ===
ABS_allocation = 95.07  # % in CLO/ABS
MMF_allocation = 2.34   # % in MMF or other

# === Define rating mapping (Moody's and S&P) to 1–4 scale ===
def map_agency_rating(r):
    r = str(r).strip().upper()
    # Moody's
    if r in ['AAA', 'AAA', 'AAA', 'AAA', 'AAA', 'AAA']: return 1
    if r in ['AA1', 'AA2', 'AA3', 'AA+', 'AA', 'AA-']: return 1
    if r in ['A1', 'A2', 'A3', 'A+', 'A', 'A-']: return 2
    if r in ['BAA1', 'BAA2', 'BAA3', 'BBB+', 'BBB', 'BBB-']: return 3
    if r in ['BA1', 'BA2', 'BA3', 'BB+', 'BB', 'BB-', 'B+', 'B', 'B-', 'C', 'CA', 'CAA', 'CCC+', 'CCC', 'CCC-', 'CC', 'D']: return 4
    return np.nan

# === Load Excel file ===
file_path = "CLOZ_CLP.xlsx"
df = pd.read_excel(file_path)

# Rename for consistency
df.columns = ['Security', '%Net', 'BVAL', 'CCC', 'Default', 'ManualRating', 'Rating_Agency']

# Score functions
def bval_score(x):
    if x >= 97.5: return 1
    elif x >= 95: return 2
    elif x >= 92: return 3
    else: return 4

def ccc_score(x):
    if x < 5: return 1
    elif x <= 7.5: return 2
    elif x <= 10: return 3
    else: return 4

def default_score(x):
    if x < 0.1: return 1
    elif x < 0.5: return 2
    elif x < 1.0: return 3
    else: return 4


# Score base parameters
df['BVAL_Score'] = df['BVAL'].apply(lambda x: bval_score(x) if not pd.isna(x) else np.nan)
df['CCC_Score'] = df['CCC'].apply(lambda x: ccc_score(x) if not pd.isna(x) else np.nan)
df['Default_Score'] = df['Default'].apply(lambda x: default_score(x) if not pd.isna(x) else np.nan)

# Score agency rating
df['Agency_Score'] = df['Rating_Agency'].apply(map_agency_rating)

# Composite score (4-factor)
def compute_composite(row):
    # If all three base scores are missing → use ManualRating (for ETF-in-ETF)
    if pd.isna(row['BVAL_Score']) and pd.isna(row['CCC_Score']) and pd.isna(row['Default_Score']):
        return row['ManualRating']
    else:
        return (
            0.3 * row['CCC_Score'] +
            0.3 * row['BVAL_Score'] +
            0.2 * row['Default_Score'] +
            0.2 * row['Agency_Score']
        )

df['Composite_Score'] = df.apply(compute_composite, axis=1)

# Risk category
def risk_category(score):
    if score <= 1.75: return 'Benign'
    elif score <= 2.5: return 'Average'
    elif score <= 3.25: return 'Stressed'
    else: return 'Crisis'

df['Category'] = df['Composite_Score'].apply(risk_category)

# Normalize weights
df['Normalized_Weight'] = df['%Net'] / df['%Net'].sum()

# ETF score (CLO-only holdings)
etf_clo_score = (df['Normalized_Weight'] * df['Composite_Score']).sum()

# Adjusted ETF score (includes MMF, assumed benign)
etf_adjusted_score = (
    etf_clo_score * (ABS_allocation / 100) +
    1.0 * (MMF_allocation / 100)
)
etf_rating = risk_category(etf_adjusted_score)

# Final Output
category_counts = df['Category'].value_counts().reindex(['Benign', 'Average', 'Stressed', 'Crisis']).fillna(0).astype(int)

print("CLOs per Category:")
print(category_counts)

print(f"\nUnadjusted CLO-only ETF Score: {etf_clo_score:.2f}")
print(f"Adjusted ETF Score (with MMF): {etf_adjusted_score:.2f}")
print(f"Final ETF Rating: {etf_rating}")


CLOs per Category:
Category
Benign      2
Average     8
Stressed    7
Crisis      3
Name: count, dtype: int32

Unadjusted CLO-only ETF Score: 2.58
Adjusted ETF Score (with MMF): 2.48
Final ETF Rating: Average


In [11]:
import pandas as pd
import numpy as np

# === CONFIGURE ETF ALLOCATION PERCENTAGES ===
ABS_allocation = 95.07  # % in CLO/ABS
MMF_allocation = 2.34   # % in MMF or other

# === Define rating mapping (Moody's and S&P) to 1–4 scale ===
def map_agency_rating(r):
    r = str(r).strip().upper()
    # Moody's
    if r in ['AAA', 'AAA', 'AAA', 'AAA', 'AAA', 'AAA']: return 1
    if r in ['AA1', 'AA2', 'AA3', 'AA+', 'AA', 'AA-']: return 1
    if r in ['A1', 'A2', 'A3', 'A+', 'A', 'A-']: return 2
    if r in ['BAA1', 'BAA2', 'BAA3', 'BBB+', 'BBB', 'BBB-']: return 3
    if r in ['BA1', 'BA2', 'BA3', 'BB+', 'BB', 'BB-', 'B+', 'B', 'B-', 'C', 'CA', 'CAA', 'CCC+', 'CCC', 'CCC-', 'CC', 'D']: return 4
    return np.nan

# === Load Excel file ===
file_path = "ICLO_CLP.xlsx"
df = pd.read_excel(file_path)

# Rename for consistency
df.columns = ['Security', '%Net', 'BVAL', 'CCC', 'Default', 'ManualRating', 'Rating_Agency']

# Score functions
def bval_score(x):
    if x >= 97.5: return 1
    elif x >= 95: return 2
    elif x >= 92: return 3
    else: return 4

def ccc_score(x):
    if x < 5: return 1
    elif x <= 7.5: return 2
    elif x <= 10: return 3
    else: return 4

def default_score(x):
    if x < 0.1: return 1
    elif x < 0.5: return 2
    elif x < 1.0: return 3
    else: return 4


# Score base parameters
df['BVAL_Score'] = df['BVAL'].apply(lambda x: bval_score(x) if not pd.isna(x) else np.nan)
df['CCC_Score'] = df['CCC'].apply(lambda x: ccc_score(x) if not pd.isna(x) else np.nan)
df['Default_Score'] = df['Default'].apply(lambda x: default_score(x) if not pd.isna(x) else np.nan)

# Score agency rating
df['Agency_Score'] = df['Rating_Agency'].apply(map_agency_rating)

# Composite score (4-factor)
def compute_composite(row):
    # If all three base scores are missing → use ManualRating (for ETF-in-ETF)
    if pd.isna(row['BVAL_Score']) and pd.isna(row['CCC_Score']) and pd.isna(row['Default_Score']):
        return row['ManualRating']
    else:
        return (
            0.3 * row['CCC_Score'] +
            0.3 * row['BVAL_Score'] +
            0.2 * row['Default_Score'] +
            0.2 * row['Agency_Score']
        )

df['Composite_Score'] = df.apply(compute_composite, axis=1)

# Risk category
def risk_category(score):
    if score <= 1.75: return 'Benign'
    elif score <= 2.5: return 'Average'
    elif score <= 3.25: return 'Stressed'
    else: return 'Crisis'

df['Category'] = df['Composite_Score'].apply(risk_category)

# Normalize weights
df['Normalized_Weight'] = df['%Net'] / df['%Net'].sum()

# ETF score (CLO-only holdings)
etf_clo_score = (df['Normalized_Weight'] * df['Composite_Score']).sum()

# Adjusted ETF score (includes MMF, assumed benign)
etf_adjusted_score = (
    etf_clo_score * (ABS_allocation / 100) +
    1.0 * (MMF_allocation / 100)
)
etf_rating = risk_category(etf_adjusted_score)

# Final Output
category_counts = df['Category'].value_counts().reindex(['Benign', 'Average', 'Stressed', 'Crisis']).fillna(0).astype(int)

print("CLOs per Category:")
print(category_counts)

print(f"\nUnadjusted CLO-only ETF Score: {etf_clo_score:.2f}")
print(f"Adjusted ETF Score (with MMF): {etf_adjusted_score:.2f}")
print(f"Final ETF Rating: {etf_rating}")


CLOs per Category:
Category
Benign      12
Average      8
Stressed     0
Crisis       0
Name: count, dtype: int32

Unadjusted CLO-only ETF Score: 1.59
Adjusted ETF Score (with MMF): 1.53
Final ETF Rating: Benign


In [13]:
import pandas as pd
import numpy as np

# === CONFIGURE ETF ALLOCATION PERCENTAGES ===
ABS_allocation = 95.07  # % in CLO/ABS
MMF_allocation = 2.34   # % in MMF or other

# === Define rating mapping (Moody's and S&P) to 1–4 scale ===
def map_agency_rating(r):
    r = str(r).strip().upper()
    # Moody's
    if r in ['AAA', 'AAA', 'AAA', 'AAA', 'AAA', 'AAA']: return 1
    if r in ['AA1', 'AA2', 'AA3', 'AA+', 'AA', 'AA-']: return 1
    if r in ['A1', 'A2', 'A3', 'A+', 'A', 'A-']: return 2
    if r in ['BAA1', 'BAA2', 'BAA3', 'BBB+', 'BBB', 'BBB-']: return 3
    if r in ['BA1', 'BA2', 'BA3', 'BB+', 'BB', 'BB-', 'B+', 'B', 'B-', 'C', 'CA', 'CAA', 'CCC+', 'CCC', 'CCC-', 'CC', 'D']: return 4
    return np.nan

# === Load Excel file ===
file_path = "JAAA_CLP.xlsx"
df = pd.read_excel(file_path)

# Rename for consistency
df.columns = ['Security', '%Net', 'BVAL', 'CCC', 'Default', 'ManualRating', 'Rating_Agency']

# Score functions
def bval_score(x):
    if x >= 97.5: return 1
    elif x >= 95: return 2
    elif x >= 92: return 3
    else: return 4

def ccc_score(x):
    if x < 5: return 1
    elif x <= 7.5: return 2
    elif x <= 10: return 3
    else: return 4

def default_score(x):
    if x < 0.1: return 1
    elif x < 0.5: return 2
    elif x < 1.0: return 3
    else: return 4


# Score base parameters
df['BVAL_Score'] = df['BVAL'].apply(lambda x: bval_score(x) if not pd.isna(x) else np.nan)
df['CCC_Score'] = df['CCC'].apply(lambda x: ccc_score(x) if not pd.isna(x) else np.nan)
df['Default_Score'] = df['Default'].apply(lambda x: default_score(x) if not pd.isna(x) else np.nan)

# Score agency rating
df['Agency_Score'] = df['Rating_Agency'].apply(map_agency_rating)

# Composite score (4-factor)
def compute_composite(row):
    # If all three base scores are missing → use ManualRating (for ETF-in-ETF)
    if pd.isna(row['BVAL_Score']) and pd.isna(row['CCC_Score']) and pd.isna(row['Default_Score']):
        return row['ManualRating']
    else:
        return (
            0.3 * row['CCC_Score'] +
            0.3 * row['BVAL_Score'] +
            0.2 * row['Default_Score'] +
            0.2 * row['Agency_Score']
        )

df['Composite_Score'] = df.apply(compute_composite, axis=1)

# Risk category
def risk_category(score):
    if score <= 1.75: return 'Benign'
    elif score <= 2.5: return 'Average'
    elif score <= 3.25: return 'Stressed'
    else: return 'Crisis'

df['Category'] = df['Composite_Score'].apply(risk_category)

# Normalize weights
df['Normalized_Weight'] = df['%Net'] / df['%Net'].sum()

# ETF score (CLO-only holdings)
etf_clo_score = (df['Normalized_Weight'] * df['Composite_Score']).sum()

# Adjusted ETF score (includes MMF, assumed benign)
etf_adjusted_score = (
    etf_clo_score * (ABS_allocation / 100) +
    1.0 * (MMF_allocation / 100)
)
etf_rating = risk_category(etf_adjusted_score)

# Final Output
category_counts = df['Category'].value_counts().reindex(['Benign', 'Average', 'Stressed', 'Crisis']).fillna(0).astype(int)

print("CLOs per Category:")
print(category_counts)

print(f"\nUnadjusted CLO-only ETF Score: {etf_clo_score:.2f}")
print(f"Adjusted ETF Score (with MMF): {etf_adjusted_score:.2f}")
print(f"Final ETF Rating: {etf_rating}")


CLOs per Category:
Category
Benign      10
Average     10
Stressed     0
Crisis       0
Name: count, dtype: int32

Unadjusted CLO-only ETF Score: 1.67
Adjusted ETF Score (with MMF): 1.61
Final ETF Rating: Benign


In [15]:
import pandas as pd
import numpy as np

# === CONFIGURE ETF ALLOCATION PERCENTAGES ===
ABS_allocation = 95.07  # % in CLO/ABS
MMF_allocation = 2.34   # % in MMF or other

# === Define rating mapping (Moody's and S&P) to 1–4 scale ===
def map_agency_rating(r):
    r = str(r).strip().upper()
    # Moody's
    if r in ['AAA', 'AAA', 'AAA', 'AAA', 'AAA', 'AAA']: return 1
    if r in ['AA1', 'AA2', 'AA3', 'AA+', 'AA', 'AA-']: return 1
    if r in ['A1', 'A2', 'A3', 'A+', 'A', 'A-']: return 2
    if r in ['BAA1', 'BAA2', 'BAA3', 'BBB+', 'BBB', 'BBB-']: return 3
    if r in ['BA1', 'BA2', 'BA3', 'BB+', 'BB', 'BB-', 'B+', 'B', 'B-', 'C', 'CA', 'CAA', 'CCC+', 'CCC', 'CCC-', 'CC', 'D']: return 4
    return np.nan

# === Load Excel file ===
file_path = "JBBB_CLP.xlsx"
df = pd.read_excel(file_path)

# Rename for consistency
df.columns = ['Security', '%Net', 'BVAL', 'CCC', 'Default', 'ManualRating', 'Rating_Agency']

# Score functions
def bval_score(x):
    if x >= 97.5: return 1
    elif x >= 95: return 2
    elif x >= 92: return 3
    else: return 4

def ccc_score(x):
    if x < 5: return 1
    elif x <= 7.5: return 2
    elif x <= 10: return 3
    else: return 4

def default_score(x):
    if x < 0.1: return 1
    elif x < 0.5: return 2
    elif x < 1.0: return 3
    else: return 4


# Score base parameters
df['BVAL_Score'] = df['BVAL'].apply(lambda x: bval_score(x) if not pd.isna(x) else np.nan)
df['CCC_Score'] = df['CCC'].apply(lambda x: ccc_score(x) if not pd.isna(x) else np.nan)
df['Default_Score'] = df['Default'].apply(lambda x: default_score(x) if not pd.isna(x) else np.nan)

# Score agency rating
df['Agency_Score'] = df['Rating_Agency'].apply(map_agency_rating)

# Composite score (4-factor)
def compute_composite(row):
    # If all three base scores are missing → use ManualRating (for ETF-in-ETF)
    if pd.isna(row['BVAL_Score']) and pd.isna(row['CCC_Score']) and pd.isna(row['Default_Score']):
        return row['ManualRating']
    else:
        return (
            0.3 * row['CCC_Score'] +
            0.3 * row['BVAL_Score'] +
            0.2 * row['Default_Score'] +
            0.2 * row['Agency_Score']
        )

df['Composite_Score'] = df.apply(compute_composite, axis=1)

# Risk category
def risk_category(score):
    if score <= 1.75: return 'Benign'
    elif score <= 2.5: return 'Average'
    elif score <= 3.25: return 'Stressed'
    else: return 'Crisis'

df['Category'] = df['Composite_Score'].apply(risk_category)

# Normalize weights
df['Normalized_Weight'] = df['%Net'] / df['%Net'].sum()

# ETF score (CLO-only holdings)
etf_clo_score = (df['Normalized_Weight'] * df['Composite_Score']).sum()

# Adjusted ETF score (includes MMF, assumed benign)
etf_adjusted_score = (
    etf_clo_score * (ABS_allocation / 100) +
    1.0 * (MMF_allocation / 100)
)
etf_rating = risk_category(etf_adjusted_score)

# Final Output
category_counts = df['Category'].value_counts().reindex(['Benign', 'Average', 'Stressed', 'Crisis']).fillna(0).astype(int)

print("CLOs per Category:")
print(category_counts)

print(f"\nUnadjusted CLO-only ETF Score: {etf_clo_score:.2f}")
print(f"Adjusted ETF Score (with MMF): {etf_adjusted_score:.2f}")
print(f"Final ETF Rating: {etf_rating}")


CLOs per Category:
Category
Benign       2
Average     10
Stressed     8
Crisis       0
Name: count, dtype: int32

Unadjusted CLO-only ETF Score: 2.30
Adjusted ETF Score (with MMF): 2.21
Final ETF Rating: Average
